In [5]:
from ucb import LinUCB
from scripts import evaluate_policy, model
from oracle import LinearOracle, OnlineRandomForestOracle, NeuralOracle, GPOracle, AdaptiveOracle
from ucb import NeuralUCB, NeuralTS, LinUCB
import torch

In [6]:
import torch
import torch.nn as nn

class FCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=2, dropout_prob=0.25):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.net(x)

In [15]:
import pandas as pd
import pickle

offline_model = pickle.load(open('fcnn_model.pkl', 'rb'))

df = pd.read_csv('data/stroke_data_2000.csv')

In [16]:
import pandas as pd
df = pd.read_csv('data/stroke_data_2000.csv')

context = df.drop(columns=['ohs6', 'itt_treat']).values
action = df['itt_treat'].values
reward = df['ohs6'].values

context = torch.tensor(context.astype(float))
action = torch.tensor(action.astype(float))
reward = torch.tensor(reward.astype(float))

data_stream = list(zip(context, action, reward))

In [17]:
lin_ucb_policy = LinUCB(context.shape[1], 1.0)

In [19]:
import numpy as np
import torch

offline_model(torch.tensor(np.concatenate([context[0], [1.]]).astype(float), dtype=torch.float32).view(1, -1))

offline_model(torch.tensor(np.concatenate([context[0], [0.]]).astype(float), dtype=torch.float32).view(1, -1))

tensor([[nan, nan, nan, nan, nan, nan, nan]], grad_fn=<AddmmBackward0>)

In [29]:
# Main experimental loop
possible_actions = [0, 1]
results = {}
for oracle in [LinearOracle, OnlineRandomForestOracle, NeuralOracle, GPOracle]:
    for policy in [lin_ucb_policy]:
        cumulative_reward = 0
        
        for t, (context, true_action, true_reward) in enumerate(data_stream):
            # Select action
            action = policy.select_action(context, possible_actions)
            
            # Get reward (true or imputed)
            if action == true_action:
                reward = true_reward
            else:
                reward = offline_model(torch.tensor(np.concatenate([context, [action]]).astype(float), dtype=torch.float32).view(1, -1))
            
            # Update policy
            policy.update(context, action, reward)
            cumulative_reward += reward
            
        results[(oracle, policy)] = cumulative_reward

RuntimeError: The size of tensor a (56) must match the size of tensor b (2) at non-singleton dimension 1

In [30]:
reward

tensor([[nan, nan]], grad_fn=<AddmmBackward0>)